In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sio
import sys
import yaml

sys.path.append("../")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from preprocessing.skoda_preprocess import get_train_val_test
from model.hierarchical_self_attention_model import HSA_model_session_guided_window

In [4]:
hyperparameters = yaml.load(open('../configs/metadata.yaml', mode='r'), Loader=yaml.FullLoader)

metadata_file = open('../configs/metadata.yaml', mode='r')

In [9]:
skoda_file = '/home/hariub/data/HAR/raw/SkodaMiniCP_2015_08/right_classall_clean.mat'

data_dict = sio.loadmat(skoda_file, squeeze_me=True)
all_data = data_dict[list(data_dict.keys())[3]]

In [18]:
x_train, y_train, x_test, y_test, x_val, y_val = get_train_val_test(all_data)

In [19]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(571774, 60)
(571774,)
(63535, 60)
(63535,)
(70595, 60)
(70595,)


In [20]:
import json
import os
activity_map = json.load(open(os.path.join('..','data', 'activity_maps', 'skoda_activity.json')))

In [21]:
activity_map

{'0': 'null',
 '1': 'write on notepad',
 '2': 'open hood',
 '3': 'close hood',
 '4': 'check gaps on the front door',
 '5': 'open left front door',
 '6': 'close left front door',
 '7': 'close both left door',
 '8': 'check trunk gaps',
 '9': 'open and close trunk',
 '10': 'check steering wheel'}